# ZUMI MAIN (cleaned)

In [ ]:
from zumi.zumi import Zumi
import time
import re

import csv
from zumi.util.screen import Screen
from zumi.util.camera import Camera
from zumi.util.vision import Vision
from zumi.personality import Personality
from datetime import datetime

from zumi.protocol import Note


zumi = Zumi()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()

screen.happy()

#GLOBAL VARs ------ EINGABEN PARAMETER HIER - NICHT MEHR BEI AUFRUF FUNCTION ---------------------------------------------------------------------

whiteTreshold = 150 # keine Änderung notwendig (Boden IR Sensoren)
circRounds = 1 # Anzahl Runden im Circle (LEft/Right)
speed = 15
jump_back_threshold = 15 # leicht reflexionsabhängig (10 - gut am abend, 20 gut bei Tag)

# VARS LOGGING ----------

face_count = 0

counter_left = 0  
counter_right = 0
counter_object = 0

# ------------------------------------------------------------------------------------------------------------------------------------------------
# Helper func Spurcorr
def corr_line(heading):
    head = heading
    ir_readings = zumi.get_all_IR_data()
    left_ir_bottom = ir_readings[3]
    right_ir_bottom = ir_readings[1]

    if left_ir_bottom > whiteTreshold and right_ir_bottom <= whiteTreshold:
        head += 5
    elif right_ir_bottom > whiteTreshold and left_ir_bottom <= whiteTreshold:
        head -= 5
    elif left_ir_bottom <= whiteTreshold and right_ir_bottom <= whiteTreshold:
        zumi.stop()
        head = "weiss"

    return head

# Helper func   
def smooth(raw_values):
    smoothed = []
    for i in range(len(raw_values)):
        if i < 2:  # Handle first two elements
            window = raw_values[0:i+1]
        else:
            window = raw_values[i-2:i+1]
        avg = sum(window) / len(window)
        smoothed.append(avg)
    return smoothed


# Func "sucht" eckige Kurve wenn auf weiss fährt am ende schwarz linie
def search_curve(zumi):
    global counter_left # von ST aus loggin func csv ------ muss getestet werden mit integration
    global counter_right # von ST aus loggin func csv

    zumi.stop()
    print("Linie verloren – starte gezielte Suche...")

    zumi.reset_gyro()
    time.sleep(0.1)

    # === Suche nach Westen/links (90 Grad) ===
    zumi.signal_left_on()
    zumi.turn(90)
    time.sleep(0.3)
    zumi.signal_left_off()
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Westen gefunden.")
        zumi.signal_left_on() # von ST ------ muss getestet werden mit integration
        counter_left +=1 # von ST aus loggin func csv ------ muss getestet werden mit integration
        log_aktion(f"{counter_left}. Links Kurve")  # von ST aus loggin func csv ------ muss getestet werden mit integration
        zumi.signal_left_off()# von ST ------ muss getestet werden mit integration
        zumi.reset_gyro()
        return 0
    if left_ir_bottom > whiteTreshold:
        # Versuch Feinjustierung nach links
        print("Linker Sensor sieht Linie – feinjustiere weiter nach links...")
        zumi.signal_left_on()  # von ST  ------ muss getestet werden mit integration
        zumi.turn_left(20)
        time.sleep(0.3)
        zumi.signal_left_off() # von ST------ muss getestet werden mit integration
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im westen gefunden.")
            zumi.reset_gyro()
            return 0

    # === Suche nach Osten/rechts (-90 Grad) ===
    zumi.signal_right_on()  # von ST  ------ muss getestet werden mit integration
    zumi.turn(-90)  # -90 zurück + 20 zusätzliche Korrekturrichtung
    time.sleep(0.3)
    zumi.signal_right_off() # von ST  ------ muss getestet werden mit integration
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Osten gefunden.")
        counter_right += 1 # von ST aus loggin func csv ------ muss getestet werden mit integration
        log_aktion(f"{counter_right}. Rechts Kurve") # von ST aus loggin func csv ------ muss getestet werden mit integration
        zumi.reset_gyro()
        return 0
    if right_ir_bottom > whiteTreshold:
        # Feinjustierung weiter nach rechts
        print("Rechter Sensor sieht Linie – feinjustiere weiter nach rechts...")
        zumi.signal_right_on() # von ST  ------ muss getestet werden mit integration
        zumi.turn_right(20)
        time.sleep(0.3)
        zumi.signal_right_off() # von ST  ------ muss getestet werden mit integration
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im Osten gefunden.")
            zumi.reset_gyro()
            return 0

    # === Keine Linie gefunden ===
    print("Keine Linie erkannt.")
    return None

# Eckiger Kreisverkehr (unabhängig von Grösse)
def follow_circle(zumi, direction="right"):
    heading = 0
    turn_count = 0
    max_turns = 4 * circRounds
    rev = "right"

    while True:
        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – Turn QR Circle richtung...")

            if direction == "right":
                zumi.turn(-90)  # -90 zurück
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "left"

            elif direction == "left":
                zumi.turn(90)  # -90 zurück
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "right"

            turn_count += 1
            print("Abbiegevorgang:", turn_count, "von", max_turns)

            if new_heading is not None:
                heading = new_heading
                if turn_count >= max_turns:
                    while True:
                        print("LOOP AFTER CIRCLE")
                        heading = corr_line(heading)
                        if heading == "weiss":
                            print("Weiße Fläche erkannt")
                            if rev == "left":
                                zumi.turn(90)
                                zumi.reset_gyro()
                            elif rev == "right":
                                zumi.turn(-90)
                                zumi.reset_gyro()
                            return 0
                        
                        zumi.go_straight(10, heading)  # langsam geradeaus
                        time.sleep(0.005)
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
        
        print("HEADING CIRCLE FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)

    zumi.stop()
    
# Grundfunc linienverfolgung
def follow_line(zumi):
    heading = 0
    start_time = time.time()
    print("HEADING START: ", heading)

    # Historie mit Startwerten füllen
    ir_readings = zumi.get_all_IR_data()
    right_ir_history = [ir_readings[0]] * 10
    left_ir_history = [ir_readings[5]] * 10

    while True:
        ir_readings = zumi.get_all_IR_data()
        left_ir_front = ir_readings[5]
        right_ir_front = ir_readings[0]

        # Historie aktualisieren
        right_ir_history.append(right_ir_front)
        left_ir_history.append(left_ir_front)
        if len(right_ir_history) > 10:
            print(right_ir_history)
            right_ir_history.pop(0)
        if len(left_ir_history) > 10:
            print(left_ir_history)
            left_ir_history.pop(0)

        # noise filtering
        right_smoothed = smooth(right_ir_history)
        left_smoothed = smooth(left_ir_history)

        # Kumulierte Deltas berechnen (using smoothed values)
        delta_right_total = sum([
            abs(right_smoothed[i+1] - right_smoothed[i])
            for i in range(len(right_smoothed)-1)
        ])
        delta_left_total = sum([
            abs(left_smoothed[i+1] - left_smoothed[i])
            for i in range(len(left_smoothed)-1)
        ])

        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – suche neue Richtung...")
            new_heading = search_curve(zumi)
            if new_heading is not None:
                heading = new_heading
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
                
        # === 2. Reagiere auf kumulativen Sprung ===
        if delta_left_total > jump_back_threshold or delta_right_total > jump_back_threshold:
            zumi.stop()
            print("KUMULATIVER IR-Sprung erkannt! ΔL=" + str(delta_left_total) + " ΔR=" + str(delta_right_total))
            object_ahead(zumi, screen, personality)
            # Historie mit Startwerten füllen
            print("HEADING KUM: ", heading)
            
            ### History cleared
            ir_readings = zumi.get_all_IR_data()
            right_ir_history.clear()
            left_ir_history.clear()
            right_ir_history = [ir_readings[0]] * 10
            left_ir_history = [ir_readings[5]] * 10
            zumi.reset_gyro()
            heading = 0
            print("HEADING NEW KUM: ", heading)
            continue
        
        print("HEADING FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)

    zumi.stop()

# Func zum lesen QR Code
def read_qr_message(image):
    qr_code = vision.find_QR_code(image)
    if qr_code is not None:
        message = vision.get_QR_message(qr_code)
        print("QR-Code erkannt:", message)
        return message
    else:
        return None

# Func Ausführen Action QR Code    
def handle_qr_action(message, zumi, screen, personality):
    emotions = ["Zumi is happy today!", "Zumi is angry today!", "Zumi is celebrating today!"]
    directions = ["Turn Right", "Turn Left", "Right Circle", "Left Circle"]

# COPY FEATURE REQUEST START ---------------------------------  (nicht tested, SG 13.05.2025)

    feature_task = ["360 nach Links", "360 nach rechts"]

    if any(keyword in message for keyword in feature_task):
        print("Fahre langsam vor zur Kreuzung...")
        
        match = re.search(r'(\d+)\s*x?\s*360', message)
        if match:
            num_turns = int(match.group(1))  # Extrahierte Zahl (z. B. "2" aus "2x 360 nach links")
        else:
            num_turns = 1  # Falls keine Zahl angegeben ist, gehe von 1 Drehung aus

        while True:
            ir_readings = zumi.get_all_IR_data()
            left_ir_bottom = ir_readings[3]
            right_ir_bottom = ir_readings[1]

            # Schwellenwert z. B. < 100 = weiß erkannt
            if left_ir_bottom <= whiteTreshold and right_ir_bottom <= whiteTreshold:
                zumi.stop()
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, 0)  # langsam geradeaus
            time.sleep(0.05)
        if "nach Links" in message and "360-" in message:
            print(f"QR-Aktion: {num_turns} mal 360 Grad nach links drehen")
            for _ in range(num_turns):
                zumi.turn_left(360, 3)
            zumi.forward(2)

        elif "nach rechts" in message and "360-" in message:
            print(f"QR-Aktion: {num_turns} mal 360 Grad nach rechts drehen")
            for _ in range(num_turns):
                zumi.turn_right(360, 3)
            zumi.forward(2)

# COPY FEATURE REQUEST END ---------------------------------

    if message in directions:
        print("Fahre langsam vor zur Kreuzung...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        if message == "Turn Right":
            print("QR-Aktion: Rechts abbiegen")
            zumi.turn_right(-90)

        elif message == "Turn Left":
            print("QR-Aktion: Links abbiegen")
            zumi.turn_left(90)


        elif message == "Right Circle":
            print("QR-Aktion: Rechtskreis")
            follow_circle(zumi, direction="right")

        elif message == "Left Circle":
            print("QR-Aktion: Linkskreis")
            follow_circle(zumi, direction="left")


    elif message in emotions:
        print("Fahre langsam zur Zielzone...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break
            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Zielfläche erreicht – führe Emotion aus")
                break

            zumi.go_straight(10, head)
            time.sleep(0.05)

        if message == "Zumi is happy today!":
            print("QR-Aktion: Freude anzeigen")
            screen.happy()
            personality.happy()

        elif message == "Zumi is angry today!":
            print("QR-Aktion: Wut anzeigen")
            screen.angry()
            personality.angry()

        elif message == "Zumi is celebrating today!":
            print("QR-Aktion: Feier-Modus")
            personality.celebrate()

        print("Zumi ist am Ziel angekommen und bleibt stehen.")
        while True:
            zumi.stop()
            time.sleep(1)  # Endlosschleife = vollständiger Stopp


    elif message == "Stop":
        print("QR-Aktion: Stop – Zumi hält an")
        zumi.stop()
        time.sleep(1.5) # hier einstellen wie lange gestoppt wird
        return

    # wurde entfernt aus den missions
    #elif message == "Park":
    #    None
    # wurde entfernt aus den missions
    #elif message == "Drive Round Road":
    #    None

    else:
        print("QR-Inhalt unbekannt:", message)
        
    time.sleep(0.5)  # kleine Pause nach der Aktion

# Func stoppen for QR oder Object (plötzlich/fliessend)
def object_ahead(zumi, screen, personality):
    global counter_object # von ST aus loggin func ------muss getestet werden  ------> achtung hier änderung der logik - neue log meth notwendig vermutlich 
    zumi.stop()
    time.sleep(0.7)

    ir_readings = zumi.get_all_IR_data()
    base_left_ir = ir_readings[5]
    base_right_ir = ir_readings[0]

    print("Hindernis erkannt – Stop & Foto")
    zumi.play_note(Note.C4, 500)
    zumi.play_note(Note.C5, 500)  # Spielt mittleres C für 500ms
    zumi.brake_lights_on()
    screen.angry()
    personality.angry()
    time.sleep(0.7)
    zumi.brake_lights_off()
    print("IR vorne: IR0 =", base_right_ir, "IR5 =", base_left_ir)

    try:
        log_aktion("Zumi startet Kamera") # ST aus logg function ---- testen
        camera.start_camera()
        time.sleep(0.9)
        image = camera.capture()
        camera.show_image(image)
        message = read_qr_message(image)
        time.sleep(1.0)

        if message is not None:
            print("QR erkannt – führe Aktion aus")
            handle_qr_action(message, zumi, screen, personality)
            time.sleep(1.0)
        else:
            print("Kein QR-Code – warte auf Objekt-Entfernung...")
            log_aktion("Zumi erkennt ein Hindernis und bleibt stehen") # von ST -----testen 

            # Historie für Differenzsprung rückwärts (FIFO-Puffer)
            left_ir_history = [base_left_ir] * 10
            right_ir_history = [base_right_ir] * 10

            time.sleep(0.5)

            while True:
                ir_readings = zumi.get_all_IR_data()
                left_ir = ir_readings[5]
                right_ir = ir_readings[0]

                left_ir_history.append(left_ir)
                right_ir_history.append(right_ir)
                if len(left_ir_history) > 10:
                    left_ir_history.pop(0)
                if len(right_ir_history) > 10:
                    right_ir_history.pop(0)

                # Kumulierte Differenzen berechnen (über Zwischenwerte)
                delta_left_total = sum([
                    left_ir_history[i+1] - left_ir_history[i]
                    for i in range(len(left_ir_history)-1)
                ])
                delta_right_total = sum([
                    right_ir_history[i+1] - right_ir_history[i]
                    for i in range(len(right_ir_history)-1)
                ])

                print("IR: L=" + str(left_ir) + ", R=" + str(right_ir) +
                      " | ΔL=" + str(delta_left_total) + ", ΔR=" + str(delta_right_total))

                if abs(delta_left_total) > jump_back_threshold or abs(delta_right_total) > jump_back_threshold:
                    print("Kumulativer IR-Sprung (Rückgang) erkannt – prüfe auf Gesicht.")
                    #break
                    time.sleep(0.5)
    
                    # Gesichtserkennung nach Objektentfernung
                    print("Prüfe auf Gesicht hinter dem Objekt...")
                    image = camera.capture()
                    camera.show_image(image)
                    face_coords = vision.find_face(image)

                    if face_coords is not None:
                        print("Gesicht erkannt!")
                        #global face_count
                        #face_count += 1
                        screen.draw_text_center("Gesicht erkannt!")
                    else:
                        print("Kein Gesicht erkannt.")

                    break

                time.sleep(0.5)

    finally:
        screen.happy()
        camera.close()
        log_aktion("Zumi schliesst die Camera") # von ST ----testen 
        print("Kamera geschlossen.")
        time.sleep(0.5)

    return True


# Ini Kombi follow line (inkl. Zeitbegrenzung)
def drive_course(zumi, total_time=60):

    print("Starte Parkour-Fahrt")
    start_time = time.time()

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")

        # Linien folgen für festgelegte Zeit
        follow_line(zumi)

    zumi.stop()
    print("Parkour abgeschlossen.")

### Aufruf main func

In [ ]:
# Zeitbegrenzung noch auf alt 120 sec = 2 mins, -> neu = 6 mins -> 360
drive_course(zumi, total_time=120)

### Notbremse nach Interrupt

In [ ]:
zumi.stop()

# Ablage Csv loggin func  

Da vermutlich so nicht funktionierendem im code (änderungen object ahead etc) - hab ich aber nicht getestet (SG, 13.05.2025)

In [ ]:
# Func zum starten (total time auf contest eingestellt -> 2 mins)
zumi = Zumi()
zumi.mpu.calibrate_MPU()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()
drive_course(zumi, screen, personality, total_time=120, block_duration=6, speed=15, threshold=150, ir_threshold_left=170, ir_threshold_right=180, treshold_white_stop = 100)

## CSV Datei erstellen und schreiben

In [ ]:
# Beim Start einen Dateinamen mit Datum und Uhrzeit erstellen
def create_log_file():
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = "CSV_von_Roberto_Sabina_Silas_Yessin_" + timestamp + ".csv"
    filepath = os.path.join("logs", filename)

    # Ordner "logs" erstellen, falls er nicht existiert
    os.makedirs("logs", exist_ok=True)

    # Neue Datei mit Header anlegen
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Zeit", "Aktion"])

    return filepath

logfile = create_log_file()

# Aktion in die aktuelle Logdatei schreiben
def log_aktion(filepath = create_log_file(), aktion = "Aktion nicht angegeben"):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filepath, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([now, aktion])



In [ ]:
###### ALTE INIT FUNC von ST 

def drive_course(zumi, screen, personality, total_time=60, block_duration=5, speed=30, threshold=150, ir_threshold_left=170, ir_threshold_right=180, treshold_white_stop = 100):
    global logfile
    global counter_right
    global counter_left
    global counter_object
    if logfile is None:
        logfile = create_log_file()
    
    if counter_right != 0 or counter_left != 0 or counter_object != 0:
        counter_left = 0
        counter_right = 0
        counter_object = 0
    print("Starte Parkour-Fahrt mit QR-Logik...")
    start_time = time.time()
    log_aktion("Das Zumi fährt los")

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")

        # Linien folgen für festgelegte Zeit
        #screen, personality sind neu für object ahead
        #treshold left, right und white sind auch neu um die werte zu definieren
        follow_line(zumi, screen, personality,  duration=block_duration, speed=speed, threshold=threshold, ir_threshold_left=ir_threshold_left, ir_threshold_right=ir_threshold_right, treshold_white_stop = treshold_white_stop)

    zumi.stop()
    end_time = time.time()
    log_aktion(f"Das Zumi hat den Parkour abgeschlossen die dafür benötigte Zeit = {start_time - end_time}")
    print("Parkour abgeschlossen.")
